In [23]:
# import API keys
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a.env file
_ = load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

# Load opensource api models
groql_api_key = os.environ["GROQ_API_KEY"]

In [25]:
# import Groq chat model
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough


llamaChatModel = ChatGroq(
    model="llama3-70b-8192"
)

chain = RunnablePassthrough()

In [26]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [27]:
prompt = ChatPromptTemplate.from_template("tell me something curious about a {soccer_player}")

output_parser = StrOutputParser()

In [6]:
# Invoke the chains
chain = prompt | llamaChatModel | output_parser

chain.invoke({"soccer_player" : "Mbappe"})

'Here\'s something curious about Kylian Mbappé:\n\n**Did you know that Kylian Mbappé is a huge fan of basketball and has a special bond with NBA legend LeBron James?**\n\nIn fact, Mbappé has often spoken about how he draws inspiration from James\' work ethic, dedication, and philanthropic efforts. The two have exchanged jerseys and have been spotted together at various sporting events.\n\nWhat\'s even more fascinating is that Mbappé has been known to incorporate some basketball-inspired moves into his soccer games. He has been seen performing "crossover" style dribbles, similar to those used in basketball, to beat opponents on the pitch!\n\nThis unique connection between two of the world\'s top athletes from different sports is certainly an interesting curiosity about Mbappé\'s personality and interests outside of soccer.'

### .bind() Runnables Function
* To add arguments to a Runnable functionin aLCEL chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [28]:
chain = prompt | llamaChatModel.bind(stop=["Ronaldo"]) | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

"Here's something curious about Cristiano "

### Combining LECL Chains

In [29]:
llamaChatModel


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027EA1E89390>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027EA1EAEA10>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********'))

In [11]:
prompt = ChatPromptTemplate.from_template("Tell me a sentence about {politician}")

chain = prompt | llamaChatModel | output_parser

In [12]:
chain.invoke("Chamberlain")

"Neville Chamberlain, the Prime Minister of the United Kingdom, infamously signed the Munich Agreement in 1938, allowing Nazi Germany to annex Czechoslovakia's Sudetenland."

### Combined Chain

In [30]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for humanity")

composed_chain = {"politician": chain} | historian_prompt | llamaChatModel | output_parser

In [17]:
composed_chain.invoke(" Chamberlain")

"What a complex and controversial figure Neville Chamberlain is!\n\nWhile Chamberlain's policy of appeasement towards Nazi Germany, particularly with regards to the Munich Agreement in 1938, is widely criticized and considered a grave mistake, it's essential to evaluate his actions in the context of the time and consider both the positive and negative aspects of his premiership.\n\n**Negative aspects:**\n\n1. **Appeasement policy**: Chamberlain's decision to give in to Hitler's demands, allowing Nazi Germany to annex Czechoslovakia's Sudetenland, is seen as a catastrophic mistake. It emboldened Hitler, who continued to pursue an aggressive expansionist policy, ultimately leading to the outbreak of World War II.\n2. **Munich Agreement**: The agreement, signed on September 30, 1938, is often viewed as a symbol of Britain's weakness and failure to stand up to Nazi aggression. It allowed Germany to occupy Czechoslovakia, which led to the country's eventual dismemberment.\n3. **Failure to p

### Another Example: A chain inside a chain

In [31]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | llamaChatModel
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

"Une question très facile !\n\nLe pays dont François Mitterrand a été le Président de 1981 à 1995 est la France, et la France est située sur le continent européen.\n\nDonc, la réponse est : l'Europe."

### LCEL chain at work in a typical RAG app

In [36]:
from langchain import hub

In [40]:
import bs4
from langchain import hub

from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Picking content from an online github source repository
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

# Automatically created from Langchain HUB online 
#prompt = hub.pull("rlm/rag-prompt")
prompt = hub.pull("rlm/rag-prompt-llama")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model     #openai model
    | StrOutputParser()
)

c:\Users\memor\anaconda3\envs\llmapp\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [39]:
rag_chain.invoke("What is Task Decomposition?")

"Task Decomposition involves breaking down complicated tasks into smaller and simpler steps. This technique, such as the Chain of Thought (CoT), helps enhance model performance on complex tasks by transforming them into multiple manageable tasks. By decomposing hard tasks, agents can better interpret the model's thinking process and achieve better results."